In [2]:
import numpy as np
from scipy.stats import norm
from scipy.stats import uniform
from scipy.stats import gamma
from scipy.stats import expon
from scipy.stats import binom



Designing scoring rules sampling on gaussian distribution

In [3]:
#
data_normal = norm.rvs(size=1000, loc=0, scale = 1)

# Creating a couple potential distributions to compare with
normal1 = norm(loc=0, scale = 1)
normal2 = norm(loc=1, scale = 1)
normal3 = norm(loc=0, scale = 2)
binom1 = binom(n=2,p=0.5)


#print(data_normal)
print(normal1.rvs(10))

[ 0.30178721 -0.15417147  0.11322084 -1.12857724 -2.08445771 -0.05350785
 -0.80609694  1.18467787  0.80817859  0.35986619]


### Energy Score
$\textnormal{Energy Score} (F,x) = 2E_F[||X-x||] - E_F[||X-X'||]$


In [4]:
def energyscore1(samples, predist):

    # sample to true 
    M=samples.size
    distsamples = predist.rvs(M)

    E1 = sum(abs(distsamples - samples))/M
    #pairs of samples to itself
    E2 = np.sum(abs(distsamples - distsamples.reshape(-1,1)))/(M*M)  ##This line is bad storage wise

    score = 2*E1 - E2
    print(score)
    return(score)

In [5]:
## Testing on different distributions
data_normaltest = norm.rvs(size=1000, loc=1, scale = 2)

#
energyscore1(data_normaltest, normal1)
energyscore1(data_normaltest, normal2)
energyscore1(data_normaltest, normal3)
energyscore1(data_normaltest, binom1)

#distribution closest does have lowest energy score


2.876083731930875
2.4028325709409586
2.4059928029109394
2.6043920571658337


np.float64(2.6043920571658337)